This notebook grades transfer survey questions and runs descriptive analysis on these grades to inform future analysis.

In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import re
from __future__ import division  #needed for eval to divide fractions to float instead of integer default of Python v<3
from utils_read_parsing import *

%matplotlib inline
matplotlib.rcParams['figure.figsize'] = 10,4
CMAP = ['cornflowerblue','#fc8d62','#a6d854','#b3b3b3','#e78ac3','#66c2a5','#ffd92f']
matplotlib.rcParams['axes.color_cycle'] = CMAP
plt.style.use = 'default'
plt.rcParams['figure.figsize'] = (8, 6)
pd.options.display.max_columns = 999

d:\applications\anaconda2\lib\site-packages\matplotlib\__init__.py:913: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


# Import files

Loading post survey data.

In [2]:
try:
    posts = pd.read_csv('post_survey_results.txt', sep='\t')
except:
    raise Exception("Try running the notebook data_log_to_pre_post_surveys_connector.ipynb and then run this cell again.")
posts.replace('Absorbance','L',inplace=True)
posts.replace('Capacitance','C',inplace=True)
posts.head(2)

,Unnamed: 0,age,debug1,debug2,english.0-writing,english.1-reading,[gender] To which gender do you most identify? [Gender non conforming/non-binary],[gender] To which gender do you most identify? [Man],[gender] To which gender do you most identify? [Prefer not to answer],gender-Woman,original id,major,open1,open2,perceivedvalue.0-boring,perceivedvalue.1-productive,perceivedvalue.2-useless,perceivedvalue.3-engaging,pocc.0-learning the basic concepts,pocc.1-testing my ideas and theories,pocc.2-answering given questions,pocc.3-memorizing key information,pocc.4-exploring the topic,qual2,qual3,qual4,qual5,qual5reasoning,qual6,qual6reasoning,quant.0,quant.0.1,quant.1,quant.1.1,quant.2,quant.2.1,quant.3,quant.3.1,strategies.0-setup my first experiment right away,strategies.10-try to identify mathematical relationships,strategies.11-try to apply my prior knowledge of physics,strategies.12-predict the result of a test before trying something out,strategies.13-decide what to investigate based on previous results,strategies.14-identify what I haven't investigated yet,strategies.1-spend time designing a set of experiments I want to conduct,strategies.2-make a plan,strategies.3-explore the different components of the PhET interface,strategies.4-identify important variables,strategies.5-look for weird outcomes I did not expect,strategies.6-look for general patterns and then dive into detailed experimentation,strategies.7-check if the relationships I found hold under different conditions,strategies.8-think about why I get the results I get,strategies.9-summarize my findings as I go,strategies_att.0-try to get the right answer,strategies_att.1-improve my scientific reasoning skill,strategies_att.2-try to learn more about the topic,strategies_att.3-finish as quickly as possible,strategies_att.4-come up with my own questions to investigate,taskinterpretation.0-investigate the basic mechanics of the topic at hand,taskinterpretation.1-design my own experiments that can help me understand the topic at hand,taskinterpretation.2-memorize information about the topic at hand,taskinterpretation.3-complete a certain number of questions,taskinterpretation.4-develop scientific reasoning skills,[year] What year are you in? [1st year undergraduate],[year] What year are you in? [2nd year undergraduate],[year] What year are you in? [3rd year undergraduate],[year] What year are you in? [4th year undergraduate],sim,sim_index,qual1,sid
0,2,18-19,"Based on virtual lab results, the charge vs. p...",The negative should not be there because it on...,Fluent,Fluent,NaN,NaN,NaN,1.0,77047160,*Non science or applied science major,I conducted 4 different experiments where I va...,Graphs helped visualize the patterns. Next tim...,Somewhat,Mostly,Not at all,Mostly,Often,Almost always,Almost always,Often,Often,higher voltage battery (https://survey.ubc.ca/...,(https://survey.ubc.ca/media/assets/user/2316...,smaller separation (https://survey.ubc.ca/medi...,(https://survey.ubc.ca/media/assets/user/2316...,The magnitude of the voltage is increasing in ...,two large plates perfectly aligned (https://su...,The plates need to be aligned perfectly in ord...,NaN,0.5,NaN,0.25,NaN,2,NaN,08-Jan,Mostly,Definitely,Somewhat,Mostly,Mostly,Mostly,Mostly,Somewhat,Mostly,Somewhat,Definitely,Mostly,Definitely,Definitely,Definitely,Definitely,Mostly,Somewhat,Somewhat,Mostly,Mostly,Mostly,Somewhat,Mostly,Mostly,1.0,NaN,NaN,NaN,C,2,larger plates (https://survey.ubc.ca/media/ass...,77047160
1,3,20-22,"k1, k2, and k3 are unnecessary. There should b...","s^2 should be s, as Q is inversely proportiona...",Average,Fluent,NaN,1.0,NaN,NaN,90447168,*Undeclared,I tried to explore more different combinations...,I believe I managed to find all the possible o...,Not at all,Mostly,Not at all,Mostly,Almost always,Often,Almost always,Almost always,Often,higher voltage battery (https://survey.ubc.ca/...,(https://survey.ubc.ca/media/assets/user/2316...,smaller separation (https://survey.ubc.ca/medi...,(https://su

Loading answerkey for quantitative and qualitative transfer questions

In [23]:
df_ans = pd.read_csv('all_quant_answers_key.csv', delimiter=',')
qual_colnames = [c for c in set(df_ans['question']) if 'qual' in c]
quant_L_colnames = [c for c in set(df_ans[df_ans['sim']=='L']['question']) if 'quant' in c]
quant_C_colnames = [c for c in set(df_ans[df_ans['sim']=='C']['question']) if 'quant' in c]
df_ans.head()

,sim,question,answer,score
0,L,quant.0,0,0
1,L,quant.1,0,0
2,L,quant.2,0,0
3,L,quant.3,0,0
4,C,quant.3.1,0,0


# Grade questions

In [27]:
def grade_questions( df, df_ans, sim, questions,  id_header ):
    students = df[id_header]
    for question in questions:
        for student in students:
            idx = df[(df[id_header]==student)&(df['sim']==sim)].index.tolist()[0]
            student_ans = df.loc[idx][question]
#             if "Jan" in student_ans:
#                 student_ans = '1/' + student_ans[1]
#                 print student_ans
            try:
                score = df_ans.get_value(df_ans[(df_ans['answer']==student_ans)&(df_ans['question']==question)&(df_ans['sim']==sim)].index.tolist()[0],'score')
                df.set_value(idx, question + '_score', score)
            except:
                print "Couldn't find grarde for:", sim, student, student_ans, question

In [25]:
grade_questions(posts, df_ans, 'L', quant_L_colnames, 'sid')
grade_questions(posts, df_ans, 'C', quant_C_colnames, 'sid')

In [26]:
grade_questions(posts, df_ans, 'L', qual_colnames, 'sid')
grade_questions(posts, df_ans, 'C', qual_colnames, 'sid')

## Renaming columns to facilitate analysis

In [ ]:
#We can rename questions given what we know about them
new_col_names = {'qual1_L_score':'qual1_L_near_concentration',
             'qual2_L_score':'qual2_L_far_volume',
             'qual3_L_score':'qual3_L_near_width',
             'qual4_L_score':'qual4_L_near_detector',
             'qual5_L_score':'qual5_L_far_laserangle',
             'qual6_L_score':'qual6_L_near_wavelength',
             'qual1_C_score':'qual1_C_near_area',
             'qual2_C_score':'qual2_C_near_voltage',
             'qual3_C_score':'qual3_C_far_areazerovoltage',
             'qual4_C_score':'qual4_C_near_separation',
             'qual5_C_score':'qual5_C_near_polarvoltage',
             'qual6_C_score':'qual6_C_far_offset'}

posts.rename(index=str,columns=new_col_names,inplace=True)

In [ ]:
qual_score_columns = ['qual1_L_near_concentration', #near transfer absorbance
                      'qual3_L_near_width',
                      'qual4_L_near_detector',
                      'qual6_L_near_wavelength',
                      #
                      'qual4_C_near_separation', #near transfer capacitance
                      'qual1_C_near_area',
                      'qual2_C_near_voltage',
                      'qual5_C_near_polarvoltage',
                      #
                      'qual5_L_far_laserangle', #far transfer
                      'qual2_L_far_volume',
                      'qual3_C_far_areazerovoltage',
                      'qual6_C_far_offset']
qual_scores = pd.concat([posts[[col for col in qual_score_columns if 'L' in col]].dropna().reset_index(drop=True),posts[[col for col in qual_score_columns if 'C' in col]].dropna().reset_index(drop=True)],axis=1)
qual_scores.head()

# Descriptive analysis

## Qualitative grades

In [ ]:
qual_scores.apply(lambda x: x.value_counts()).transpose().iloc[::-1,:].plot.barh(stacked=True)
ax = plt.subplot()
ax.legend(loc=2,frameon=True,labels=['correct','incorrect']);ax.set_xlabel('Percent of students',fontsize=13)
ax.set_xticklabels([0,20,40,60,80,100],fontsize=13)
ax.set_xticks([round(int(x)*148.0/100.0,1) for x in [0,20,40,60,80,100]])
ax.set_title("Stacked bar chart of percent of students who got a qualitative question correct",fontsize=15);

In [ ]:
from scipy.stats import spearmanr
from sklearn.metrics import jaccard_similarity_score
correlation_matrix = np.zeros((len(qual_score_columns),len(qual_score_columns)))
for i,qual_scores_i in enumerate(qual_score_columns):
    for j,qual_scores_j in enumerate(qual_score_columns):
        jc = jaccard_similarity_score(list(qual_scores[qual_scores_i].dropna().astype(int)),list(qual_scores[qual_scores_j].dropna().astype(int)))
        correlation_matrix[i,j] = jc
#         print att_i,att_j,r,p

np.fill_diagonal(correlation_matrix,0)
# correlation_matrix[(correlation_matrix<0.4)&(correlation_matrix>-0.4)] = 0
sns.heatmap(correlation_matrix,yticklabels=qual_score_columns,xticklabels=qual_score_columns,annot=True);

In [ ]:
from scipy.stats import pearsonr
def cronbach_alpha(scores):
    K = scores.shape[1]
    sum_of_var = scores.apply(lambda x: np.var(x), axis=0).sum()
    var = np.var(scores.sum(axis=1))
    alpha = (K/(K-1))*(1-sum_of_var/var)
    return round(alpha,1)

def spearman_brown_coef(scores):
    ''' Use for reliability of two item scale
    https://link.springer.com/content/pdf/10.1007/s00038-012-0416-3.pdf
    '''
    r,p = pearsonr(scores.ix[:,0],scores.ix[:,1])
    coef = 2*r/(1+r)
    return round(coef,1)

print 'all (12)',cronbach_alpha(qual_scores)
print 'near variables (4)',cronbach_alpha(qual_scores[['qual1_L_near_concentration','qual3_L_near_width','qual4_C_near_separation','qual1_C_near_area']])
print 'near not variables (4)',cronbach_alpha(qual_scores[['qual4_L_near_detector','qual6_L_near_wavelength','qual2_C_near_voltage','qual5_C_near_polarvoltage']])
print 'near absorbance (4)',cronbach_alpha(qual_scores[['qual4_L_near_detector','qual6_L_near_wavelength','qual1_L_near_concentration','qual3_L_near_width']])
print 'near capacitance (4)',cronbach_alpha(qual_scores[['qual4_C_near_separation','qual1_C_near_area','qual2_C_near_voltage','qual5_C_near_polarvoltage']])
print 'near all (8)',cronbach_alpha(qual_scores[['qual1_L_near_concentration','qual3_L_near_width','qual4_C_near_separation','qual1_C_near_area','qual4_L_near_detector','qual6_L_near_wavelength','qual2_C_near_voltage','qual5_C_near_polarvoltage']])
print 'far all (4)',cronbach_alpha(qual_scores[['qual5_L_far_laserangle','qual2_L_far_volume','qual3_C_far_areazerovoltage','qual6_C_far_offset']])

print 'near variable absorbance (2)', jaccard_similarity_score(qual_scores['qual1_L_near_concentration'],qual_scores['qual3_L_near_width'])
print 'near variable capacitance (2)', jaccard_similarity_score(qual_scores['qual4_C_near_separation'],qual_scores['qual1_C_near_area'])
print 'far variable absorbance (2)', jaccard_similarity_score(qual_scores['qual5_L_far_laserangle'],qual_scores['qual2_L_far_volume'])
print 'far variable capacitance (2)', jaccard_similarity_score(qual_scores['qual3_C_far_areazerovoltage'],qual_scores['qual6_C_far_offset'])

## Quantitative grades

In [ ]:
posts[posts['sim']=='L'][[c for c in posts.columns if 'score' in c]].hist()

In [ ]:
# # print(dfL.describe())  #basic descriptive stats for each column
# # dfL.hist(bins=3)  #plots set of histograms for all columns
# # dfL.groupby('condition').hist(bins=3)  #plots 2 sets of histograms of each column, one set for each condition
# # histogram of one column grouped by condition and overlayed
# dfL.groupby('condition').quant1_score.plot.hist(bins=3,alpha=0.4,legend=True)  
# # dfL.groupby('condition').count()
# # dfL.groupby('condition').transform(lambda x: x / x.count())  #useful snippet to manipulate column data per condition

# # making histograms directly using numpy/matplotlib for full control, e.g. normalizing frequency so to sum=1
# # x1 = dfL[dfL['condition']=='CL']['quant1_score']
# # x2 = dfL[dfL['condition']=='LC']['quant1_score']
# # hist1, bins1 = np.histogram(x1, bins=3)
# # hist2, bins2 = np.histogram(x2, bins=3)
# # plt.bar(bins1[:-1], hist1 / hist1.sum(), width=(bins1[1]-bins1[0]), color='red',alpha=0.5, label='CL')
# # plt.bar(bins2[:-1], hist2 / hist2.sum(), width=(bins2[1]-bins2[0]), color='blue',alpha=0.5, label='LC')
# # plt.legend(loc='upper left')